In [49]:
import os 
import joblib
## Data manipulation
import pandas as pd
import numpy as np

## Modeling
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


from sklearn.tree import DecisionTreeClassifier

## visuatlization
import plotly 
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
# from datalearn19intro import (get_accounts, get_events, get_subscriptions, get_users)
# acc = get_accounts()
# events = get_events()
# subs = get_subscriptions()
# users = get_users()

In [13]:
## todo relative path 
data = joblib.load(os.path.join(os.getcwd(), 'Data','preprocessed_data.jblib'))

# Split to train test

In [14]:
y = data['lead_score']
X = data.loc[:,(data.columns != 'lead_score') & (data.columns != 'account_id')]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [19]:
## check if classes ratio is the same in train and test
ratio_train = y_train.sum() / (1-y_train).sum()
ratio_test = y_test.sum() / (1-y_test).sum()
print(ratio_train, ",", ratio_test)

0.025493504226839648 , 0.025813845217204454


# Logistic Regression

# NN